In [2]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
from clin.config import PATH_REPO
import clin.modules.ebm.extract
import clin.modules.ebm.omission
import clin.modules.ebm.prune
import clin.modules.ebm.evidence
import clin.eval.ebm
import clin.eval.eval
from clin.modules import ebm
import joblib

df = joblib.load(join(PATH_REPO, 'data', 'ebm', 'ebm_interventions_cleaned.pkl'))
df = df.iloc[:100]
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]
n = len(dfe)
llm = clin.llm.get_llm('text-davinci-003')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
r = {}
i = 0
extractor = ebm.extract.Extractor()
n_shots = 5
interventions = extractor(i, df, nums, n_shots, llm)

In [4]:
ov = ebm.omission.OmissionVerifier()
pv = ebm.prune.PruneVerifier()
ev = ebm.evidence.EvidenceVerifier()
r["list_original"] = [
    extractor(i, df, nums, n_shots, llm) for i in tqdm(range(len(df)))
]

r["list_ov"] = [
    ov(dfe.iloc[i]["doc"], bullet_list=r["list_original"][i], llm=llm)
    for i in tqdm(range(n))
]


# r["list_pv"] = [
#     pv(dfe.iloc[i]["doc"], bullet_list=r["list_original"][i], llm=llm)
#     for i in tqdm(range(n))
# ]

r["list_ov_pv"] = [
    pv(dfe.iloc[i]["doc"], bullet_list=r["list_ov"][i], llm=llm) for i in tqdm(range(n))
]

# r["dict_evidence_ov_pv_ev"] = [
#     ev(dfe.iloc[i]["doc"], bullet_list=r["list_ov_pv"][i], llm=llm)
#     for i in tqdm(range(n))
# ]
# r["list_ov_pv_ev"] = [list(r["dict_evidence_ov_pv_ev"][i].keys()) for i in range(n)]

100%|██████████| 100/100 [00:00<00:00, 9626.81it/s]


# Evaluate

In [8]:
# add metrics to r
ks_list = [k for k in r.keys() if k.startswith("list_")]
for k in ks_list:
    mets_df = pd.DataFrame(
        [
            clin.eval.eval.calculate_precision_recall_from_lists(
                *clin.eval.ebm.process_ebm_lists(r[k][i], dfe.iloc[i]["interventions"]),
                verbose=False,
            )
            for i in range(len(dfe))
        ]
    )
    mets_dict_single = clin.eval.eval.aggregate_precision_recall(mets_df)
    for k_met in mets_dict_single.keys():
        r[k_met + "___" + k] = mets_dict_single[k_met]



# show metrics
row_df = pd.DataFrame(
pd.Series({k: r[k] for k in r.keys() if "___" in k})
    .round(3)
    # .style.format(precision=3)
    # .background_gradient(cmap="Blues")
).T
# row_df = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')]
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': '',
    'level_1': 'Verifiers',
}).pivot_table(index='Verifiers', columns='', values=0).round(3)
rc.style.format(precision=3).background_gradient(cmap='Blues')

,f1,precision,recall
Verifiers,,,
list_original,0.465,0.446,0.485
list_ov,0.442,0.375,0.539
list_ov_pv,0.445,0.379,0.539


In [9]:
# compare lists
# l1 = r['list_ov']
l1 = r['list_original']
# l1 = r['list_ov_pv_ev']
l2 = r['list_ov']
for i in range(len(l1)):
    if l1[i] != l2[i]:
        print(dfe.iloc[i]['doc'])
        print(i)
        print(l1[i])
        print(l2[i])
        print()

Oral contraceptive use is associated with increased cardiovascular reactivity in nonsmokers.

Women who smoke and take oral contraceptives (OCs) have significantly increased risk of cardiovascular disease, but the exact mechanismsfor the increased risk are not known. Cardiovascular reactivity to psychological stress may be one mechanism for the enhanced risk, but the small number of studies examining whether OC users who smoke have greater reactivity have produced mixed results. The purpose of this study was to examine the effect of chronic cigarette smoking, acute nicotine administration, and OC use on cardiovascular and lipid reactivity. Sixty healthy women, half of whom had been using OCs for at least the previous 6 months, participated in the study. Approximately two thirds were smokers and were randomized to be tested after either a 12-hr nicotine deprivation or administration of nicotine gum. One third were nonsmokers. Heart rate, blood pressure, and lipid measures were taken at 

## Look at validation data

In [ ]:
dfv = joblib.load(join(PATH_REPO, 'data', 'ebm', 'ebm_interventions_cleaned.pkl')).iloc[100:]
for i in range(len(dfv)):
    row = dfv.iloc[i]
    print(row['doc'])
    print(clin.parse.list_to_bullet_str(row['interventions']))
    print()